In [1]:
%matplotlib notebook

In [2]:
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage.io as io

In [32]:
mask = nb.load('/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight_200um.nii.gz').get_data()
aff = nb.load('/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight_200um.nii.gz').affine
hdr = nb.load('/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight_200um.nii.gz').header
cortex = np.where(mask==1)

In [21]:
df = pd.read_csv("/home/julia/data/gradients/genes/gene_list_full.csv")
df = df[df['acronym']!="FALSE"]

In [39]:
gene_matrix = np.zeros((cortex[0].shape[0], len(df)))
ids = list(df['id'])
for n in range(len(ids)):
    try:
        gene = np.swapaxes(io.imread('/home/julia/data/gradients/genes/energy/%s/energy.mhd'% ids[n],
                                     plugin='simpleitk'), 
                           0,2)[:-1,:-1,:-1]
        nb.save(nb.Nifti1Image(gene, aff, hdr), '/home/julia/data/gradients/genes/energy/%s/energy.nii.gz'% ids[n])
        gene_matrix[:,n] = gene[cortex]
    except RuntimeError:
        pass

In [42]:
np.save('/home/julia/data/gradients/genes/gene_matrix.npy', gene_matrix)